In [1]:
from IPython.display import HTML
HTML(open('../style.css').read())

# Implementing an LR-Table-Generator

## A Grammar for Grammars

As the goal is to generate an *SLR-table-generator* we first need to implement a parser for context free grammars.
The file `arith.g` in the directory `Examples` contains an example grammar that describes arithmetic expressions.

In [2]:
!cat Examples/arith.g

expr: expr '+' product
    | expr '-' product
    | product
    ;
 
product: product '*' factor
       | product '/' factor
       | factor
       ;
       
factor: '(' expr ')'
      | NUMBER
      ;


We use <span style="font-variant:small-caps;">Ply</span> to develop a parser for context free grammars.  

In [3]:
import ply.lex as lex

In [4]:
tokens = [ 'VARIABLE',  # r'[a-z][a-z0-9_]*'
           'TOKEN',     # r'[A-Z][A-Z0-9_]*'
           'LITERAL',   # r"'.'"
         ]

In [5]:
t_VARIABLE = r'[a-z][a-z0-9_]*'
t_TOKEN    = r'[A-Z][A-Z0-9_]*'

In [6]:
def t_comment(t):
    r'//.*'

In [7]:
def t_LITERAL(t):
    r"'.*?'"
    return t

In [8]:
literals = [':', '|', ';']

In [9]:
t_ignore = ' \t\r'

In [10]:
def t_newline(t):
    r'\n'
    t.lexer.lineno += 1
    return

In [11]:
def find_column(token):
    program    = token.lexer.lexdata  # the complete string given to the scanner
    line_start = program.rfind('\n', 0, token.lexpos)
    return token.lexpos - line_start

In [12]:
def t_error(t):
    column = find_column(t)
    print(f"Illegal character '{t.value[0]}' in line {t.lineno}, column {column}.")
    t.lexer.skip(1)

In [13]:
__file__ = 'main'

In [14]:
lexer = lex.lex()

In [15]:
def test_scanner(file_name):
    with open(file_name, 'r') as handle:
        program = handle.read() 
    print(program)
    lexer.input(program)
    lexer.lineno = 1          # reset line number
    for t in lexer:           # start scanning and collect all tokens
        print(t) 

In [16]:
test_scanner('Examples/arith.g')

expr: expr '+' product
    | expr '-' product
    | product
    ;
 
product: product '*' factor
       | product '/' factor
       | factor
       ;
       
factor: '(' expr ')'
      | NUMBER
      ;

LexToken(VARIABLE,'expr',1,0)
LexToken(:,':',1,4)
LexToken(VARIABLE,'expr',1,6)
LexToken(LITERAL,"'+'",1,11)
LexToken(VARIABLE,'product',1,15)
LexToken(|,'|',2,27)
LexToken(VARIABLE,'expr',2,29)
LexToken(LITERAL,"'-'",2,34)
LexToken(VARIABLE,'product',2,38)
LexToken(|,'|',3,50)
LexToken(VARIABLE,'product',3,52)
LexToken(;,';',4,64)
LexToken(VARIABLE,'product',6,68)
LexToken(:,':',6,75)
LexToken(VARIABLE,'product',6,77)
LexToken(LITERAL,"'*'",6,85)
LexToken(VARIABLE,'factor',6,89)
LexToken(|,'|',7,103)
LexToken(VARIABLE,'product',7,105)
LexToken(LITERAL,"'/'",7,113)
LexToken(VARIABLE,'factor',7,117)
LexToken(|,'|',8,131)
LexToken(VARIABLE,'factor',8,133)
LexToken(;,';',9,147)
LexToken(VARIABLE,'factor',11,157)
LexToken(:,':',11,163)
LexToken(LITERAL,"'('",11,165)
LexToken(VARIABLE,'expr',

We start by generating both scanner and parser.  

In [17]:
import ply.yacc as yacc

In [18]:
start = 'grammar'

In [19]:
def p_grammar_one(p):
    "grammar : rule"
    p[0] = p[1]

def p_grammar_more(p):
    "grammar : rule grammar"
    p[0] = p[1] + p[2]

In [20]:
def p_rule(p):
    "rule : VARIABLE ':' body_list ';'"
    p[0] = [ (p[1],) + body for body in p[3] ]

In [21]:
def p_body_list_one(p):
    "body_list : body"
    p[0] = [p[1]]

def p_body_list_more(p):
    "body_list : body '|' body_list "
    p[0] = [p[1]] + p[3]

In [22]:
def p_body_empty(p):
    "body : "
    p[0] = ()

def p_body_more(p):
    "body : item body"
    p[0] = (p[1],) + p[2]

In [23]:
def p_item_variable(p):
    "item : VARIABLE"
    p[0] = p[1]

def p_item_terminal(p):
    "item : TOKEN"
    p[0] = p[1]

def p_item_literal(p):
    "item : LITERAL"
    p[0] = p[1]

In [24]:
def p_error(t):
    column = find_column(t)
    if t:
        print(f'Syntax error at token "{t.value}" in line {t.lineno}, column {column}.')
    else:
        print('Syntax error at end of input.')

In [25]:
yacc.yacc(write_tables=False, debug=True);

Generating LALR tables


In [26]:
def parse(file):
    lexer.lineno = 1
    with open(file, 'r') as handle:
        grammar = handle.read() 
    print(grammar)
    ast = yacc.parse(grammar)
    return ast

In [27]:
ruleList = parse('Examples/arith.g')
ruleList

expr: expr '+' product
    | expr '-' product
    | product
    ;
 
product: product '*' factor
       | product '/' factor
       | factor
       ;
       
factor: '(' expr ')'
      | NUMBER
      ;



[('expr', 'expr', "'+'", 'product'),
 ('expr', 'expr', "'-'", 'product'),
 ('expr', 'product'),
 ('product', 'product', "'*'", 'factor'),
 ('product', 'product', "'/'", 'factor'),
 ('product', 'factor'),
 ('factor', "'('", 'expr', "')'"),
 ('factor', 'NUMBER')]

## The Class `GrammarRule`

The class `GrammarRule` is used to store a single grammar rule.  As we have to use objects of type `GrammarRule` as keys in a dictionary later, we have to provide the methods `__eq__`, `__ne__`, and `__hash__`.

In [28]:
class GrammarRule:
    def __init__(self, variable, body):
        self.mVariable = variable
        self.mBody     = body
        
    def __eq__(self, other):
        return isinstance(other, GrammarRule)    and \
               self.mVariable == other.mVariable and \
               self.mBody     == other.mBody
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash(self.__repr__())
    
    def __repr__(self):
        return f'{self.mVariable} → {" ".join(self.mBody)}'

The function `parse_grammar` takes a string `filename` as its argument and returns the grammar that is stored in the specified file.  The grammar is represented as list of rules.  Each rule is represented as a tuple.  The example below will clarify this structure.

In [29]:
def transform(rules):
    return [ GrammarRule(var, tuple(body)) for (var, *body) in rules]

In [30]:
grammar = transform(ruleList)
grammar

[expr → expr '+' product,
 expr → expr '-' product,
 expr → product,
 product → product '*' factor,
 product → product '/' factor,
 product → factor,
 factor → '(' expr ')',
 factor → NUMBER]

Given a string `name`, which is either a *variable*, a *token*, or a *literal*, the function `is_var` checks whether `name` is a variable.  The function can distinguish variable names from tokens and literals because variable names consist only of lower case letters, while tokens are all uppercase and literals start with the character "`'`".

In [31]:
def is_var(name):
    return name[0] != "'" and name[0].islower()

Given a list `Rules` of `GrammarRules`, the function `collect_variables(Rules)` returns the set of all *variables* occuring in `Rules`.

In [32]:
def collect_variables(Rules):
    Variables = set()
    for rule in Rules:
        Variables.add(rule.mVariable)
        for item in rule.mBody:
            if is_var(item):
                Variables.add(item)
    return Variables

Given a set `Rules` of `GrammarRules`, the function `collect_tokens(Rules)` returns the set of all *tokens* and *literals* occuring in `Rules`.

In [33]:
def collect_tokens(Rules):
    Tokens = set()
    for rule in Rules:
        for item in rule.mBody:
            if not is_var(item):
                Tokens.add(item)
    return Tokens

## Extended Marked Rules

The class `ExtendedMarkedRule` stores a single *marked rule* of the form
$$ v \rightarrow \alpha \bullet \beta : L $$
where the *variable* $v$ is stored in the member variable `mVariable`, while $\alpha$ and $\beta$ are stored in the variables `mAlpha`and `mBeta` respectively.  The set of follow tokens $L$ is stored in the variable `mFollow`. These variables are assumed to contain tuples of *grammar symbols*.  A *grammar symbol* is either
- a *variable*,
- a *token*, or
- a *literal*, i.e. a string enclosed in single quotes.


Later, we need to maintain sets of *marked rules* to represent *states*.  Therefore, we have to define the methods `__eq__`, `__ne__`, and `__hash__`.

In [34]:
class ExtendedMarkedRule():
    def __init__(self, variable, alpha, beta, follow):
        self.mVariable = variable
        self.mAlpha    = alpha
        self.mBeta     = beta
        self.mFollow   = follow
        
    def __eq__(self, other):
        return isinstance(other, ExtendedMarkedRule) and \
               self.mVariable == other.mVariable     and \
               self.mAlpha    == other.mAlpha        and \
               self.mBeta     == other.mBeta         and \
               self.mFollow   == other.mFollow
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash(self.mVariable) + \
               hash(self.mAlpha)    + \
               hash(self.mBeta)     + \
               hash(self.mFollow)
    
    def __repr__(self):
        alphaStr  = ' '.join(self.mAlpha)
        betaStr   = ' '.join(self.mBeta)
        if len(self.mFollow) > 1:
            followStr = '{' + ','.join(self.mFollow) + '}'
        else:
            followStr = ','.join(self.mFollow)
        return f'{self.mVariable} → {alphaStr} • {betaStr}: {followStr}'

Given an *extended marked rule* `self`, the function `is_complete` checks, whether the *extended marked rule* `self` has the form
$$ c \rightarrow \alpha\; \bullet: L,$$
i.e. it checks, whether the $\bullet$ is at the end of the grammar rule.

In [35]:
def is_complete(self):
    return len(self.mBeta) == 0

ExtendedMarkedRule.is_complete = is_complete
del is_complete

Given an *extended marked rule* `self` of the form
$$ c \rightarrow \alpha \bullet X\, \delta: L, $$
the function `symbol_after_dot` returns the *symbol* $X$. If there is no symbol after the $\bullet$, the method returns `None`.

In [36]:
def symbol_after_dot(self):
    if len(self.mBeta) > 0:
        return self.mBeta[0]
    return None

ExtendedMarkedRule.symbol_after_dot = symbol_after_dot
del symbol_after_dot

Given an extended marked rule, this function returns the variable following the dot.  If there is no variable following the dot, the function returns `None`.  

In [37]:
def next_var(self):
    if len(self.mBeta) > 0:
        var = self.mBeta[0]
        if is_var(var):
            return var
    return None

ExtendedMarkedRule.next_var = next_var
del next_var

The function `move_dot(self)` transforms an *extended marked rule*  of the form 
$$ c \rightarrow \alpha \bullet X\, \beta: L $$
into an *extended marked rule* of the form
$$ c \rightarrow \alpha\, X \bullet \beta: L, $$
i.e. the $\bullet$ is moved over the next symbol.  Invocation of this method assumes that there is a symbol
following the $\bullet$.

In [38]:
def move_dot(self):
    return ExtendedMarkedRule(self.mVariable, 
                              self.mAlpha + (self.mBeta[0],), 
                              self.mBeta[1:],
                              self.mFollow)

ExtendedMarkedRule.move_dot = move_dot
del move_dot

The function `to_rule(self)` turns the *extended marked rule* `self` into  a `GrammarRule`, i.e. the *extended marked rule*
$$ c \rightarrow \alpha \bullet \beta: L $$
is turned into the grammar rule
$$ c \rightarrow \alpha\, \beta. $$

In [39]:
def to_rule(self):
    return GrammarRule(self.mVariable, self.mAlpha + self.mBeta)

ExtendedMarkedRule.to_rule = to_rule
del to_rule

The function `to_rule(self)` turns the *extended marked rule* `self` into a `MarkedRule`, i.e. the *extended marked rule*
$$ c \rightarrow \alpha \bullet \beta: L $$
is turned into the marked rule
$$ c \rightarrow \alpha\bullet \beta. $$

In [40]:
def to_marked_rule(self):
    return MarkedRule(self.mVariable, self.mAlpha, self.mBeta)

ExtendedMarkedRule.to_marked_rule = to_marked_rule
del to_marked_rule

The class `MarkedRule` is similar to the class `ExtendedMarkedRule` but does not have the *follow set*.

In [41]:
class MarkedRule():
    def __init__(self, variable, alpha, beta):
        self.mVariable = variable
        self.mAlpha    = alpha
        self.mBeta     = beta
        
    def __eq__(self, other):
        return isinstance(other, MarkedRule)     and \
               self.mVariable == other.mVariable and \
               self.mAlpha    == other.mAlpha    and \
               self.mBeta     == other.mBeta     
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash(self.mVariable) + \
               hash(self.mAlpha)    + \
               hash(self.mBeta)     
    
    def __repr__(self):
        alphaStr  = ' '.join(self.mAlpha)
        betaStr   = ' '.join(self.mBeta)
        return f'{self.mVariable} → {alphaStr} • {betaStr}'

Given a set of *extended marked rules* `M`, the function `combine_rules` *combines* those extended marked ruless that have the same *core*:  If 
$$ a \rightarrow \beta \bullet \gamma : L $$
is an extended marked rule, then its *core* is defined as the marked rule
$$ a \rightarrow \beta \bullet \gamma. $$
If $a \rightarrow \beta \bullet \gamma : L_1$ and $a \rightarrow \beta \bullet \gamma : L_2$ are two extended marked rules, then they can be *combined* into the rule
$$ a \rightarrow \beta \bullet \gamma : L_1\cup L_2 $$

In [42]:
def combine_rules(M):
    Result = set()
    Core   = set()
    for emr1 in M:
        Follow = set()
        core1 = emr1.to_marked_rule()
        if core1 in Core:
            continue
        Core.add(core1)
        for emr2 in M:
            core2 = emr2.to_marked_rule()
            if core1 == core2:
                Follow |= emr2.mFollow
        new_emr = ExtendedMarkedRule(core1.mVariable, core1.mAlpha, core1.mBeta, frozenset(Follow))
        Result.add(new_emr)
    return frozenset(Result)

## LR-Table-Generation

The class `Grammar` represents a context free grammar.  It stores a list of the `GrammarRules` of the given grammar.
Each grammar rule of the form
$$ a \rightarrow \beta $$
The start symbol is assumed to be the variable on the left hand side of the first rule. The grammar is *augmented* with the rule
$$ \widehat{s} \rightarrow s. $$
Here $s$ is the start variable of the given grammar and $\widehat{s}$ is a new variable that is the start variable of the *augmented grammar*. The symbol `$` denotes the end of input.  The non-obvious member variables of the class `Grammar` have the following interpretation
- `mStates` is the set of all states of the *LR-parser*.  These states are sets of *extended marked rules*.
- `mStateNames`is a dictionary assigning names of the form `s0`, `s1`, $\cdots$, `sn` to the states stored in 
  `mStates`.  The functions `action` and `goto` will be defined for *state names*, not for *states*, because 
  otherwise the table representing these functions would become both huge and unreadable.
- `mConflicts` is a Boolean variable that will be set to true if the table generation discovers 
  *shift/reduce conflicts* or *reduce/reduce conflicts*.

In [43]:
class Grammar():
    def __init__(self, Rules):
        self.mRules      = Rules
        self.mStart      = Rules[0].mVariable
        self.mVariables  = collect_variables(Rules)
        self.mTokens     = collect_tokens(Rules)
        self.mStates     = set()
        self.mStateNames = {}
        self.mConflicts  = False
        self.mVariables.add('ŝ')
        self.mTokens.add('$')
        self.mRules.append(GrammarRule('ŝ', (self.mStart, ))) # augmenting
        self.compute_tables()

Given a set of `Variables`, the function `initialize_dictionary` returns a dictionary that assigns the empty set to all variables.

In [44]:
def initialize_dictionary(Variables):
    return { a: set() for a in Variables }

Given a `Grammar`, the function `compute_tables` computes
- the sets `First(v)` for every variable `v`,
- the set of all *states* of the *LR-Parser*,
- the *action table*, and
- the *goto table*. 

Given a grammar `g`,
- the set `g.mFirst` is a dictionary such that `g.mFirst[a] = First[a]` and
- the set `g.mFollow` is a dictionary such that `g.mFollow[a] = Follow[a]` for all variables `a`.

In [45]:
def compute_tables(self):
    self.mFirst  = initialize_dictionary(self.mVariables)
    self.compute_first()
    self.compute_rule_names()
    self.all_states()
    self.compute_action_table()
    self.compute_goto_table()
    
Grammar.compute_tables = compute_tables
del compute_tables

The function `compute_rule_names` assigns a unique name to each *rule* of the grammar.  These names are used later
to represent *reduce actions* in the *action table*.

In [46]:
def compute_rule_names(self):
    self.mRuleNames = {}
    counter = 0
    for rule in self.mRules:
        self.mRuleNames[rule] = 'r' + str(counter)
        counter += 1
        
Grammar.compute_rule_names = compute_rule_names
del compute_rule_names

The function `compute_first(self)` computes the sets $\texttt{First}(c)$ for all variables $c$ and stores them in the dictionary `mFirst`.  Abstractly, given a variable $c$ the function $\texttt{First}(c)$ is the set of all tokens that can start a string that is derived from $c$:
$$\texttt{First}(\texttt{c}) := 
  \Bigl\{ t \in T \Bigm| \exists \gamma \in (V \cup T)^*: \texttt{c} \Rightarrow^* t\,\gamma \Bigr\}.
$$
The definition of the function $\texttt{First}()$ is extended to strings from $(V \cup T)^*$ as follows:
- $\texttt{FirstList}(\varepsilon) = \{\}$.
- $\texttt{FirstList}(t \beta) = \{ t \}$  if $t \in T$.
- $\texttt{FirstList}(\texttt{a} \beta) = \left\{
       \begin{array}[c]{ll}
         \texttt{First}(\texttt{a}) \cup \texttt{FirstList}(\beta) & \mbox{if $\texttt{a} \Rightarrow^* \varepsilon$;} \\
         \texttt{First}(\texttt{a})                                & \mbox{otherwise.}
       \end{array}
       \right.
      $ 

If $\texttt{a}$ is a variable of $G$ and the rules defining $\texttt{a}$ are given as 
$$\texttt{a} \rightarrow \alpha_1 \mid \cdots \mid \alpha_n, $$
then we have
$$\texttt{First}(\texttt{a}) = \bigcup\limits_{i=1}^n \texttt{FirstList}(\alpha_i). $$
The dictionary `mFirst` that stores this function is computed via a *fixed point iteration*.

In [47]:
def compute_first(self):
    change = True
    while change:
        change = False
        for rule in self.mRules:
            a, body = rule.mVariable, rule.mBody
            first_body = self.first_list(body)
            if not (first_body <= self.mFirst[a]):
                change = True
                self.mFirst[a] |= first_body           
    print('First sets:')
    for v in self.mVariables:
        print(f'First({v}) = {self.mFirst[v]}')
        
Grammar.compute_first = compute_first
del compute_first

Given a tuple of variables and tokens `alpha`, the function `first_list(alpha)` computes the function $\texttt{FirstList}(\alpha)$ that has been defined above.  If `alpha` is *nullable*, then the result will contain the empty string $\varepsilon = \texttt{''}$.

In [48]:
def first_list(self, alpha):
    if len(alpha) == 0:
        return { '' }
    elif is_var(alpha[0]): 
        v, *r = alpha
        return eps_union(self.mFirst[v], self.first_list(r))
    else:
        t = alpha[0]
        return { t }
    
Grammar.first_list = first_list
del first_list

The arguments `S` and `T` of `eps_union` are sets that contain tokens and, additionally, they might contain the empty string.

In [49]:
def eps_union(S, T):
    if '' in S: 
        if '' in T: 
            return S | T
        return (S - { '' }) | T
    return S

Given an augmented grammar $G = \langle V,T,R\cup\{\widehat{s} \rightarrow s\,\$\}, \widehat{s}\rangle$ 
and a variable $a$, the set of tokens that might follow $a$ is defined as:
$$\texttt{Follow}(a) := 
 \bigl\{ t \in \widehat{T} \,\bigm|\, \exists \beta,\gamma \in (V \cup \widehat{T})^*: 
                           \widehat{s} \Rightarrow^* \beta \,a\, t\, \gamma 
  \bigr\}.
$$
The function `compute_follow` computes the sets $\texttt{Follow}(a)$ for all variables $a$ via a *fixed-point iteration*.

If $\mathcal{M}$ is a set of *extended marked rules*, then the *closure* of $\mathcal{M}$ is the smallest set $\mathcal{K}$ such that
we have the following:
- $\mathcal{M} \subseteq \mathcal{K}$,
- If $a \rightarrow \beta \bullet c\, \delta: L$ is a *extended marked rule* from 
  $\mathcal{K}$, $c$ is a variable, and $t\in L$ and if, furthermore,
  $c \rightarrow \gamma$ is a grammar rule,
  then the marked rule $c \rightarrow \bullet \gamma: \texttt{First}(\delta\,t)$
  is an element of $\mathcal{K}$:
  $$(a \rightarrow \beta \bullet c\, \delta:L) \in \mathcal{K} 
         \;\wedge\; 
         (c \rightarrow \gamma) \in R \;\wedge\; t \in L
         \;\Rightarrow\; (c \rightarrow \bullet \gamma: \texttt{First}(\delta\,t)) \in \mathcal{K}
  $$

We define $\texttt{closure}(\mathcal{M}) := \mathcal{K}$.  The function `cmp_closure` computes this closure for a given set of *extended marked rules* via a *fixed-point iteration*.

In [50]:
def cmp_closure(self, Marked_Rules):
    All_Rules = Marked_Rules
    New_Rules = Marked_Rules
    while True:
        More_Rules = set()
        for rule in New_Rules:
            c = rule.next_var()
            if c == None:
                continue
            delta = rule.mBeta[1:]
            L = rule.mFollow
            for newRule in self.mRules:
                head, alpha = newRule.mVariable, newRule.mBody
                if c == head:
                    newL = frozenset({ x for t in L for x in self.first_list(delta + (t,)) })
                    More_Rules |= { ExtendedMarkedRule(head, (), alpha, newL) }
        if More_Rules <= All_Rules:
            return frozenset(All_Rules)
        New_Rules  = More_Rules - All_Rules
        All_Rules |= New_Rules

Grammar.cmp_closure = cmp_closure
del cmp_closure

Given a set of *extended marked rules* $\mathcal{M}$ and a *grammar symbol* $X$, the function $\texttt{goto}(\mathcal{M}, X)$ 
is defined as follows:
$$\texttt{goto}(\mathcal{M}, X) := \texttt{closure}\Bigl( \bigl\{ 
   a \rightarrow \beta\, X \bullet \delta:L \bigm| (a \rightarrow \beta \bullet X\, \delta:L) \in \mathcal{M} 
   \bigr\} \Bigr).
$$

In [51]:
def goto(self, Marked_Rules, x):
    Result = set()
    for mr in Marked_Rules:
        if mr.symbol_after_dot() == x:
            Result.add(mr.move_dot())
    return combine_rules(self.cmp_closure(Result))

Grammar.goto = goto
del goto

The function `all_states` computes the set of all states of an *LR-parser*.  The function starts with the state
$$ \texttt{closure}\bigl(\{ \widehat{s} \rightarrow \bullet s : {\$} \}\bigr) $$
and then tries to compute new states by using the function `goto`.  This computation proceeds via a 
*fixed-point iteration*.  Once all states have been computed, the function assigns names to these states.
This association is stored in the dictionary *mStateNames*.

In [52]:
def all_states(self): 
    start_state  = self.cmp_closure({ ExtendedMarkedRule('ŝ', (), (self.mStart,), frozenset({'$'})) })
    start_state  = combine_rules(start_state)
    self.mStates = { start_state }
    New_States   = self.mStates
    while True:
        More_States = set()
        for Rule_Set in New_States:
            for mr in Rule_Set: 
                if not mr.is_complete():
                    x = mr.symbol_after_dot()
                    next_state = self.goto(Rule_Set, x)
                    if next_state not in self.mStates and next_state not in More_States:
                        More_States.add(next_state)
                        print('.', end='')
        if len(More_States) == 0:
            break
        New_States = More_States;
        self.mStates |= New_States
        print('\n', len(self.mStates), sep='')
    print("All LR-states:")
    counter = 1
    self.mStateNames[start_state] = 's0'
    print(f's0 = {set(start_state)}')
    for state in self.mStates - { start_state }:
        self.mStateNames[state] = f's{counter}'
        print(f's{counter} = {set(state)}')
        counter += 1

Grammar.all_states = all_states
del all_states

The following function computes the *action table* and is defined as follows:
- If $\mathcal{M}$ contains an *extended marked rule* of the form $a \rightarrow \beta \bullet t\, \delta:L$
  then we have
  $$\texttt{action}(\mathcal{M},t) := \langle \texttt{shift}, \texttt{goto}(\mathcal{M},t) \rangle.$$
- If $\mathcal{M}$ contains an *extended marked rule* of the form $a \rightarrow \beta\, \bullet:L$ and we have
  $t \in L$, then we define
  $$\texttt{action}(\mathcal{M},t) := \langle \texttt{reduce}, a \rightarrow \beta \rangle$$
- If $\mathcal{M}$ contains the *extended marked rule* $\widehat{s} \rightarrow s \bullet:\{\$\}$, then we define 
  $$\texttt{action}(\mathcal{M},\$) := \texttt{accept}. $$
- Otherwise, we have
  $$\texttt{action}(\mathcal{M},t) := \texttt{error}. $$

In [53]:
def compute_action_table(self):
    self.mActionTable = {}
    print('\nAction Table:')
    for state in self.mStates:
        stateName = self.mStateNames[state]
        actionTable = {}
        # compute shift actions
        for token in self.mTokens:   
            newState  = self.goto(state, token)
            if newState != set():
                newName = self.mStateNames[newState]
                actionTable[token] = ('shift', newName)
                self.mActionTable[stateName, token] = ('shift', newName)
                print(f'action("{stateName}", {token}) = ("shift", {newName})')
        # compute reduce actions
        for mr in state:
            if mr.is_complete():
                for token in mr.mFollow:
                    action1 = actionTable.get(token)
                    action2 = ('reduce', mr.to_rule())
                    if action1 == None:
                        actionTable[token] = action2  
                        r = self.mRuleNames[mr.to_rule()]
                        self.mActionTable[stateName, token] = ('reduce', r)
                        print(f'action("{stateName}", {token}) = {action2}')
                    elif action1 != action2: 
                        self.mConflicts = True
                        print('')
                        print(f'conflict in state {stateName}:')
                        print(f'{stateName} = {state}')
                        print(f'action("{stateName}", {token}) = {action1}')     
                        print(f'action("{stateName}", {token}) = {action2}')
                        print('')
        for mr in state:
            if mr == ExtendedMarkedRule('ŝ', (self.mStart,), (), frozenset({'$'})):
                actionTable['$'] = 'accept'
                self.mActionTable[stateName, '$'] = 'accept'
                print(f'action("{stateName}", $) = accept')

Grammar.compute_action_table = compute_action_table
del compute_action_table

The function `compute_goto_table` computes the *goto table*.

In [54]:
def compute_goto_table(self):
    self.mGotoTable = {}
    print('\nGoto Table:')
    for state in self.mStates:
        for var in self.mVariables:
            newState = self.goto(state, var)
            if newState != set():
                stateName = self.mStateNames[state]
                newName   = self.mStateNames[newState]
                self.mGotoTable[stateName, var] = newName
                print(f'goto({stateName}, {var}) = {newName}')

Grammar.compute_goto_table = compute_goto_table
del compute_goto_table

In [55]:
def strip_quotes(t):
    if t[0] == "'" and t[-1] == "'":
        return t[1:-1]
    return t

In [57]:
def dump_parse_table(self, file):
    with open(file, 'w', encoding="utf-8") as handle:
        handle.write('# Grammar rules:\n')
        for rule in self.mRules:
            rule_name = self.mRuleNames[rule] 
            handle.write(f'{rule_name} =("{rule.mVariable}", {rule.mBody})\n')
        handle.write('\n# Action table:\n')
        handle.write('actionTable = {}\n')
        for s, t in self.mActionTable:
            action = self.mActionTable[s, t]
            t = strip_quotes(t)
            if action[0] == 'reduce':
                rule_name = action[1]
                handle.write(f"actionTable['{s}', '{t}'] = ('reduce', {rule_name})\n")
            elif action == 'accept':
                handle.write(f"actionTable['{s}', '{t}'] = 'accept'\n")
            else:
                handle.write(f"actionTable['{s}', '{t}'] = {action}\n")
        handle.write('\n# Goto table:\n')
        handle.write('gotoTable = {}\n')
        for s, v in self.mGotoTable:
            state = self.mGotoTable[s, v]
            handle.write(f"gotoTable['{s}', '{v}'] = '{state}'\n")
        handle.write('\n# States:\n')
        handle.write('stateNames = {}\n')
        for s in self.mStateNames:
            name = self.mStateNames[s]
            handle.write(f"stateNames['{name}'] = {s}\n")           

Grammar.dump_parse_table = dump_parse_table
del dump_parse_table

In [60]:
def analyse(file):
    rules        = parse(file)
    grammarRules = transform(rules) 
    grammar      = Grammar(grammarRules)
    grammar.dump_parse_table('parse-table.py')

In [61]:
analyse('Examples/arith.g')

expr: expr '+' product
    | expr '-' product
    | product
    ;
 
product: product '*' factor
       | product '/' factor
       | factor
       ;
       
factor: '(' expr ')'
      | NUMBER
      ;

First sets:
First(product) = {'NUMBER', "'('"}
First(factor) = {'NUMBER', "'('"}
First(expr) = {'NUMBER', "'('"}
First(ŝ) = {'NUMBER', "'('"}
.....
6
.........
15
..........
25
.....
30
All LR-states:
s0 = {factor →  • '(' expr ')': {'/','*',$,'+','-'}, expr →  • expr '+' product: {'+','-',$}, product →  • product '/' factor: {'/','*',$,'+','-'}, expr →  • product: {'+','-',$}, product →  • factor: {'/','*',$,'+','-'}, product →  • product '*' factor: {'/','*',$,'+','-'}, factor →  • NUMBER: {'/','*',$,'+','-'}, ŝ →  • expr: $, expr →  • expr '-' product: {'+','-',$}}
s1 = {ŝ → expr • : $, expr → expr • '+' product: {'+','-',$}, expr → expr • '-' product: {'+','-',$}}
s2 = {factor →  • '(' expr ')': {'/','*',$,'+','-'}, product →  • product '*' factor: {'/','*',$,'+','-'}, factor →  • NU

In [ ]:
analyse('Examples/postfix.g')

In [ ]:
analyse('Examples/arith-small.g')

In [62]:
%%time
analyse('Examples/c-grammar.g')

// This is the grammar for the programming language C.

translation_unit
	: external_declaration
	| translation_unit external_declaration
	;

primary_expression
	: IDENTIFIER
	| CONSTANT
	| STRING_LITERAL
	| '(' expression ')'
	;

postfix_expression
	: primary_expression
	| postfix_expression '[' expression ']'
	| postfix_expression '(' ')'
	| postfix_expression '(' argument_expression_list ')'
	| postfix_expression '.' IDENTIFIER
	| postfix_expression '->' IDENTIFIER
	| postfix_expression '++'
	| postfix_expression '--'
	;

argument_expression_list
	: assignment_expression
	| argument_expression_list ',' assignment_expression
	;

unary_expression
	: postfix_expression
	| '++' unary_expression
	| '--' unary_expression
	| unary_operator cast_expression
	| 'sizeof' unary_expression
	| 'sizeof' '(' type_name ')'
	;

unary_operator
	: '&'
	| '*'
	| '+'
	| '-'
	| '~'
	| '!'
	;

cast_expression
	: unary_expression
	| '(' type_name ')' cast_expression
	;

multiplicative_expression
	: cast_exp

..................................
72
...................................................................................................................................................................
235
.................................................................................................................................................................................................................................................................
492
........................................................................................................................................................................................................................................................................................................................................
820
.......................................................................................................................................................................................................

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



action("s31", '{') = ("shift", s1464)
action("s31", 'while') = ("shift", s216)
action("s31", '&') = ("shift", s712)
action("s31", '(') = ("shift", s1322)
action("s31", 'default') = ("shift", s1059)
action("s31", STRING_LITERAL) = ("shift", s1254)
action("s31", 'continue') = ("shift", s1484)
action("s31", CONSTANT) = ("shift", s373)
action("s31", 'return') = ("shift", s1006)
action("s31", IDENTIFIER) = ("shift", s1002)
action("s31", '!') = ("shift", s471)
action("s31", '-') = ("shift", s728)
action("s32", '&&') = ('reduce', postfix_expression → postfix_expression '(' argument_expression_list ')')
action("s32", '>=') = ('reduce', postfix_expression → postfix_expression '(' argument_expression_list ')')
action("s32", '||') = ('reduce', postfix_expression → postfix_expression '(' argument_expression_list ')')
action("s32", '-=') = ('reduce', postfix_expression → postfix_expression '(' argument_expression_list ')')
action("s32", '%=') = ('reduce', postfix_expression → postfix_expression '('

action("s53", '{') = ("shift", s1309)
action("s53", 'const') = ("shift", s1458)
action("s53", 'while') = ("shift", s792)
action("s53", 'long') = ("shift", s352)
action("s53", '&') = ("shift", s712)
action("s53", '(') = ("shift", s1322)
action("s53", 'signed') = ("shift", s1088)
action("s53", 'static') = ("shift", s235)
action("s53", 'extern') = ("shift", s394)
action("s53", 'default') = ("shift", s985)
action("s53", 'auto') = ("shift", s65)
action("s53", 'int') = ("shift", s548)
action("s53", STRING_LITERAL) = ("shift", s1254)
action("s53", TYPE_NAME) = ("shift", s1545)
action("s53", 'continue') = ("shift", s915)
action("s53", CONSTANT) = ("shift", s373)
action("s53", 'return') = ("shift", s622)
action("s53", IDENTIFIER) = ("shift", s1497)
action("s53", '!') = ("shift", s471)
action("s53", 'short') = ("shift", s1243)
action("s53", 'typedef') = ("shift", s921)
action("s53", '-') = ("shift", s728)
action("s54", '[') = ('reduce', direct_declarator → direct_declarator '(' parameter_type_li

...

goto(s586, unary_operator) = s781
goto(s588, struct_or_union) = s1146
goto(s588, struct_or_union_specifier) = s270
goto(s588, parameter_declaration) = s549
goto(s588, type_specifier) = s92
goto(s588, enum_specifier) = s97
goto(s588, storage_class_specifier) = s228
goto(s588, type_qualifier) = s214
goto(s588, parameter_type_list) = s139
goto(s588, identifier_list) = s751
goto(s588, declaration_specifiers) = s121
goto(s588, parameter_list) = s1470
goto(s589, relational_expression) = s742
goto(s589, inclusive_or_expression) = s1016
goto(s589, primary_expression) = s234
goto(s589, conditional_expression) = s1444
goto(s589, and_expression) = s520
goto(s589, postfix_expression) = s1104
goto(s589, exclusive_or_expression) = s1380
goto(s589, equality_expression) = s1253
goto(s589, shift_expression) = s297
goto(s589, assignment_expression) = s141
goto(s589, cast_expression) = s1297
goto(s589, logical_or_expression) = s865
goto(s589, logical_and_expression) = s1418
goto(s589, additive_expression

goto(s810, unary_operator) = s698
goto(s810, multiplicative_expression) = s1087
goto(s810, specifier_qualifier_list) = s483
goto(s810, expression) = s831
goto(s811, relational_expression) = s1367
goto(s811, primary_expression) = s429
goto(s811, and_expression) = s528
goto(s811, postfix_expression) = s886
goto(s811, exclusive_or_expression) = s1490
goto(s811, equality_expression) = s1333
goto(s811, shift_expression) = s738
goto(s811, cast_expression) = s896
goto(s811, additive_expression) = s909
goto(s811, unary_expression) = s22
goto(s811, unary_operator) = s182
goto(s811, multiplicative_expression) = s815
goto(s812, primary_expression) = s234
goto(s812, postfix_expression) = s1104
goto(s812, cast_expression) = s1376
goto(s812, unary_expression) = s1161
goto(s812, unary_operator) = s126
goto(s820, struct_or_union) = s296
goto(s820, struct_or_union_specifier) = s1547
goto(s820, type_specifier) = s820
goto(s820, enum_specifier) = s24
goto(s820, storage_class_specifier) = s1533
goto(s820,

goto(s1033, expression_statement) = s414
goto(s1033, additive_expression) = s909
goto(s1033, unary_expression) = s168
goto(s1033, unary_operator) = s1271
goto(s1033, multiplicative_expression) = s815
goto(s1033, expression) = s1049
goto(s1037, relational_expression) = s467
goto(s1037, inclusive_or_expression) = s138
goto(s1037, primary_expression) = s1492
goto(s1037, and_expression) = s1186
goto(s1037, postfix_expression) = s1044
goto(s1037, exclusive_or_expression) = s119
goto(s1037, equality_expression) = s923
goto(s1037, shift_expression) = s229
goto(s1037, cast_expression) = s1227
goto(s1037, additive_expression) = s1356
goto(s1037, unary_expression) = s1026
goto(s1037, unary_operator) = s781
goto(s1037, multiplicative_expression) = s1320
goto(s1039, relational_expression) = s1367
goto(s1039, inclusive_or_expression) = s1240
goto(s1039, primary_expression) = s416
goto(s1039, conditional_expression) = s771
goto(s1039, and_expression) = s528
goto(s1039, postfix_expression) = s817
got

goto(s1309, declaration_list) = s637
goto(s1309, logical_and_expression) = s84
goto(s1309, storage_class_specifier) = s1533
goto(s1309, statement_list) = s1372
goto(s1309, expression_statement) = s1212
goto(s1309, type_qualifier) = s1022
goto(s1309, additive_expression) = s909
goto(s1309, unary_expression) = s168
goto(s1309, unary_operator) = s1271
goto(s1309, multiplicative_expression) = s815
goto(s1309, declaration_specifiers) = s34
goto(s1309, expression) = s1222
goto(s1311, relational_expression) = s1367
goto(s1311, inclusive_or_expression) = s1240
goto(s1311, primary_expression) = s416
goto(s1311, conditional_expression) = s771
goto(s1311, and_expression) = s528
goto(s1311, postfix_expression) = s817
goto(s1311, iteration_statement) = s1506
goto(s1311, exclusive_or_expression) = s408
goto(s1311, equality_expression) = s1333
goto(s1311, statement) = s538
goto(s1311, shift_expression) = s738
goto(s1311, compound_statement) = s902
goto(s1311, assignment_expression) = s477
goto(s1311,

CPU times: user 51.9 s, sys: 517 ms, total: 52.5 s
Wall time: 52.5 s
